In [1]:
%%capture
from pyspark.sql import SparkSession
import glow
spark = SparkSession \
    .builder \
    .appName("PythonPi") \
    .config("spark.jars.packages", "io.projectglow:glow-spark3_2.12:1.1.2") \
    .config("spark.hadoop.io.compression.codecs", "io.projectglow.sql.util.BGZFCodec") \
    .getOrCreate()
spark = glow.register(spark)
import pandas as pd
pd.set_option('max_columns', None)

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop2.7/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
io.projectglow#glow-spark3_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-47ded9c4-e56e-4f75-9c71-aa71c3198b7a;1.0
	confs: [default]
	found io.projectglow#glow-spark3_2.12;1.1.2 in central
	found org.seqdoop#hadoop-bam;7.9.2 in central
	found com.github.jsr203hadoop#jsr203hadoop;1.0.3 in central
	found org.slf4j#slf4j-api;1.7.25 in central
	found log4j#log4j;1.2.17 in central
	found org.slf4j#slf4j-log4j12;1.7.25 in central
	found org.jdbi#jdbi;2.63.1 in central
	found com.github.broadinstitute#picard;2.21.9 in central
	found com.intel.gkl#gkl;0.8.5 in central
	found commons-io#commons-io;2.4 in central
	found org.broadinstitute#gatk-native-bindings;0.1.0-rc-1 in central
	found org.apache.logging.log4j#log4j-api;2.5 in central
	found org.apache.logging.log4j#log4j-core;2.5 in central
	found com.google.guava#guava;15.0 in central
	

In [2]:
from pyspark.sql.functions import col, split
from pyspark.sql.types import LongType

In [26]:
%%bash
chromosome="chrY"  # Set the chromosome variable
output_file="${chromosome}_exome.vep_105.PAW.autopvs1.tsv"

zcat "/sbgenomics/project-files/${chromosome}_exome.vep_105.PAW.autopvs1.tsv.gz" | head -n 1  > "$output_file"

for FILE in "/sbgenomics/project-files/"*"${chromosome}_exome.vep_105.PAW.autopvs1.tsv.gz"; do
    # Use `zcat` to decompress and concatenate the contents of each file
    zcat "$FILE" | tail -n +2 >> "$output_file"
done

In [28]:
CHROM = 'Y'
spark.read.options(inferSchema=True,sep="\t",header=True,nullValue=".") \
    .csv("chr" + CHROM + "_exome.vep_105.PAW.autopvs1.tsv") \
    .withColumnRenamed("SYMBOL", "symbol") \
    .withColumn("split_c0", split(col("vcf_id"), "-")) \
    .withColumn("chromosome",col("split_c0").getItem(0)) \
    .withColumn("start",col("split_c0").getItem(1)) \
    .withColumn("reference",col("split_c0").getItem(2)) \
    .withColumn("alternate",col("split_c0").getItem(3)) \
    .withColumn("start", col("Start").cast(LongType())) \
    .drop("split_c0", "SYMBOL") \
    .coalesce(1) \
    .write.mode("overwrite") \
    .parquet("/sbgenomics/output-files/AutoPVS1_AutoGVP/chromosome=" + CHROM)

In [29]:
df = spark.read.parquet("/sbgenomics/output-files/AutoPVS1_AutoGVP/chromosome=Y/")
df.count()

19996560

In [65]:
from pyspark.sql.functions import col, regexp_replace, split, regexp_extract, input_file_name
from pyspark.sql.types import StructType, StructField, LongType, FloatType, DoubleType, StringType
from pyspark.sql import DataFrame
import numpy as np
import re